# Water Usage Model

In [ ]:
from epx import Job, ModelConfig, SynthPop
import json
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.io as pio
import time
import plotly.graph_objects as go
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

## Household Water Usage Depends on Demographics

The basic water usage model calculates daily household water use. First, the number of individuals (adults at least 16 years old and juveniles younger than 16) in each household is extracted. Then, the household size is used to determine the number of times each household does each activity (except for lawn watering which is done an average of once/week regardless of household size). The following activities are assumed to add to water usage: 

| Activity      | Water used (L) | Frequency |
| ----------- | ----------- |-----------|
| Lawn      | 3000       |  1/week |
| Laundry   | 70        |   1/day |
| Toilet    | 9 | 8/day |
| Shower    | 50 | 1/day |
| Eat/Drink | 8  | 5/day |
| Clean     | 3  | 1/day |


The table above shows the average amount of water used for each activity for a single individual. The total amount of water used each day is drawn from a poisson distribution with the averages shown in the table above and multiplied by household size. 

In [ ]:
water_config = ModelConfig(
    synth_pop=SynthPop("US_2010.v5", ["Kewaunee_County_WI"]),
    start_date="2022-01-01",
    end_date="2025-01-01",
)

water_job = Job(
    "model/main.fred",
    config=[water_config],
    key="cl_water_job",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results"
)

water_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 720 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(water_job.status) != 'DONE':
    if str(water_job.status) == 'ERROR':
        logs = water_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(water_job.status)

In [ ]:
household_data = water_job.results.csv_output('monthly_household_data.csv')
household_data['Date'] = pd.to_datetime(household_data['Date'], format="%Y%m%d")

In [ ]:
# single adults
single_adult = household_data.loc[(household_data['Num_Adults']==1) & (household_data['Num_Kids']==0)]
single_adult = single_adult.pivot(index='Date', columns='HH_ID', values='Household_Usage')
mu1 = single_adult.median(axis=1).rename('Single Adult')
lower1 = single_adult.quantile(axis=1, q=.25)
higher1 = single_adult.quantile(axis=1, q=.75)

# multi adult households
multi_adult = household_data.loc[(household_data['Num_Adults']>1) & (household_data['Num_Kids']==0)]
multi_adult = multi_adult.pivot(index='Date', columns='HH_ID', values='Household_Usage')
mu2 = multi_adult.median(axis=1).rename('Multi Adult')
lower2 = multi_adult.quantile(axis=1, q=.25)
higher2 = multi_adult.quantile(axis=1, q=.75)

# households with children
with_kids = household_data.loc[(household_data['Num_Adults']>0) & (household_data['Num_Kids']>0)]
with_kids = with_kids.pivot(index='Date', columns='HH_ID', values='Household_Usage')
mu3 = with_kids.median(axis=1).rename('Adults with Kids')
lower3 = with_kids.quantile(axis=1, q=.25)
higher3 = with_kids.quantile(axis=1, q=.75)

In [ ]:
medians = pd.concat([mu1, mu2, mu3], axis=1)

usage_by_comp = px.line(medians, 
                     labels={
                       "value": "Liters",
                       "variable": "Household Composition",
                     },
                     title="Total Water Usage by Households with a Given Composition"
                    )
usage_by_comp.show()

In [ ]:
bg_usage = household_data[["BG_ID", "Household_Usage"]].groupby("BG_ID").sum().reset_index().rename(columns={"Household_Usage": "Usage"})

# calculate lat, long to pass to mapbox for map center
lat_cen = 44.507970   # Kewaunee_County, WI
long_cen = -87.626833 # Kewaunee_County, WI

# set up Epistemix house map tiles
mapstyle="mapbox://styles/epxadmin/cm0ve9m13000501nq8q1zdf5p"
token="pk.eyJ1IjoiZXB4YWRtaW4iLCJhIjoiY20wcmV1azZ6MDhvcTJwcTY2YXpscWsxMSJ9._ROunfMS6hgVh1LPQZ4NGg"

# load the block group boundary file
f = open("shapefiles/Kewaunee_County_WI_blockgroups.geojson")
blocks_json = json.load(f)

# create the figure
fig2 = px.choropleth_mapbox(
    bg_usage, geojson=blocks_json,
    locations='BG_ID', color='Usage',
    color_continuous_scale=[
        (0, "rgba(40, 95, 223, 1)"), # blue
        (1, "rgba(235, 90, 54, 1)") # red
    ],
    featureidkey="properties.GEOID",
    hover_data=bg_usage.columns,
    zoom=8.5, height=700,
    center = {"lat": lat_cen,"lon": long_cen},
    labels={'Usage':'Total Usage<br>(Liters)'}
)

fig2.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)
fig2.update_layout(coloraxis_colorbar_orientation='h')
fig2.update_layout(coloraxis_colorbar_y=-0.1)
fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0.3})
fig2.show()

In [ ]:
# deleting our job now that we are done with it
water_job.delete(interactive=False)